In [1]:
## Imports
import os
import random

import torch
from tqdm.notebook import tqdm

from aics_classes import  CIFAKE_data, CIFAKE_CNN
from aics_helper import train, test, score_preds


random.seed(11)

src_dir = '../CIFAKE/'  # https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/data, Dec 11, 14:05

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <EA7F9DF5-8854-31D8-89D4-BD566CAF4DEA> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <03BFC270-EF54-36FC-B102-371F5164D3C9> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Load data

In [ ]:
# load data (my code)

def get_files(cifake_dir):
    collect = dict()
    for root, dirs, files in os.walk(cifake_dir):
        if len(files)>1:
            subdir = root.split('/')[2]
            subclass = root.split('/')[3]
            collect[subdir] = collect.get(subdir, list()) + [(os.path.join(root, fname), subclass)
                                                for fname in files]
    return collect


data = get_files(src_dir)
trainfiles = data['train']
testfiles = data['test']
trainfiles[0]  # tuple of filepath + class as str

('../CIFAKE/train/REAL/1006 (3).jpg', 'REAL')

In [3]:
# plot example pre/post normalise; cf tutorial code
# import matplotlib.pyplot as plt
# plt.imshow(np.array(Image.open(a.data[0][0])))  #normal img
# c = transform(Image.open(a.data[0][0]))  # post transform & re-normalise,,kind of
# c = c / 2 + 0.5     # unnormalize
# npimg = c.numpy()
# plt.imshow(np.transpose(npimg, (1, 2, 0)))
# plt.show()


In [ ]:
traindata = CIFAKE_data(trainfiles, batch_size=18)

## Define model

Base from AICS course cifar10_tutorial.ipynb

In [5]:
model = CIFAKE_CNN()

model = train(model, traindata, epochs=2)

 25%|██▌       | 1396/5555 [00:30<01:25, 48.55it/s]

 50%|█████     | 2787/5555 [00:57<00:50, 54.83it/s]

 75%|███████▌  | 4175/5555 [01:23<00:23, 59.53it/s]

5556it [01:49, 50.60it/s]                          


epoch: 0	total loss: 3023.6499843895435	avg loss: 0.5443114283329511


 25%|██▌       | 1399/5555 [00:26<01:14, 55.44it/s]

 50%|█████     | 2782/5555 [00:55<01:06, 41.92it/s]

 75%|███████▌  | 4174/5555 [01:24<00:25, 54.32it/s]

5556it [01:55, 48.19it/s]                          

epoch: 1	total loss: 1895.3033572994173	avg loss: 0.34118872318621374
Finished Training


In [ ]:
PATH = './model_v01.01.pth'
torch.save(model.state_dict(), PATH)

## Testing

In [ ]:
# load data
testdata = CIFAKE_data(testfiles, batch_size=18)

In [ ]:
# load model
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [7]:
# predict labels & evaluate with accuracy, precision, recall, f1-score
gold, pred = test(model, testdata)
score_preds(gold, pred)

1112it [00:18, 60.87it/s]                          


Performance
Accuracy: 87.45%
Precison: 84.67%
Recall: 91.47%
F1-Score: 87.94%
